In [121]:
!pip uninstall -y torch
!pip install transformers[torch]
!pip3 install emoji==0.6.0

Found existing installation: torch 2.1.1
Uninstalling torch-2.1.1:
  Successfully uninstalled torch-2.1.1
  Using cached torch-2.1.1-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.


In [122]:
!pip install transformers

In [123]:
# prompt: # TP4: évaluation des connaissances - IMPORTANT: Mettez votre nom et prénom dans le nom du fichier: TP4_NOM_PRENOM - IMPORTANT: Donnez moi les inputs pour lancer les 3 tâches - IMPORTANT: Donnez moi les fichiers / modèles si finetuning - IMPORTANT: Commentez votre code  ## Objectif: Créer un ChatBot permettant d'effectuer 3 tâches de NLP L'objectif de ce TP 4 est d'utiliser les connaissances collectées lors des précédents TP pour créer un ChatBot capable de comprendre et d'exécuter 3 tâches de NLP différents. Il faut 1) développer le ChatBot (sur Colab) et 2) coder 3 fonctions pour les 3 tâches de NLP.  Voici les critères d'évaluation: - Robustesse du ChatBot (vous pouvez repartir d'ELIZA et ajuster les règles) - Diversité des méthodes pour réaliser les tâches (Règles, Machine Learning, Deep Learning, Finetuning, ...) - Diversité des tâches (ne faite pas 3 fois du sentiment analysis et essayer de ne pas faire que de la classification) - Qualité du code, documentation (commentaire, instruction, explication, ...) et facilité à tester - Résultats (si finetuning)  ## Exemple d'échanges avec le ChatBot (facile) > User: sentiment-analysis: I love NLP classes!\ Bot: POSITIVE\ User: music-style: title: Flowers | author: Miley Cyrus\ Bot: Pop music\ User: ner: Apple is a big company!\ Bot: Apple -> Company  Dans cet exemple le Bot reconnaît les patterns "sentiment-analysis", "music-style" et "ner", execute les fonctions concernées et retourne les résultats à l'utilisateur.  ## Exemple d'échanges avec le ChatBot (difficile) > User: Quel est la polarité/sentiment dans "I love NLP classes!"\ Bot: Le sentiment est positif\ User: Quel est le style de cette musique: title: Flowers | author: Miley Cyrus\ Bot: C'est de la Pop music!\ User: Donne moi les entreprises dans la phrase: "Apple is a big company!"\ Bot: Il y a 1 entreprise: "Apple"  Idem que l’exemple précédent mais avec des inputs plus complexes.  ## Conseils Faites 3 fonctions f1, f2, f3 (ou, nommez les selon la tâche) et réutilisez le code de ELIZA pour déclencher les fonctions selon ce que vous détectez dans l'input utilisateur. Variez les tâches et n'hésitez pas à utiliser huggingface (https://huggingface.co/learn/nlp-course/chapter1/3?fw=pt) pour vous aider.  ## Pour ceux qui n'ont pas d'idées: - Base du ChatBot: TP1 -> ELIZA - Fonction 1: TP2 -> Text Classification - Sentiment Analysis (Machine Learning) - Fonction 2: TP3 -> Text Classification - Music style prediction (Deep Learning + Tranformer Finetuning) - Fonction 3: Doc/Cours Huggingface - IMPORTANT: MODIFIEZ UN MINIMUM LES PROJETS! Ne me rendez pas MA propre correction de TP...  ## Attention - Ne faites pas 3 fois la même tache - N'utilisez pas 3 fois la même méthode - Ne dupliquez pas le code brut d'un collègue - Utilisez ChatGPT intelligemment - Si vous utilisez des modèles non accessible en ligne, envoyez les moi! - Verifiez que le code tourne sur Colab! - Mettez votre nom et prenom dans le nom du fichier: TP4_NOM_PRENOM

import re
import pandas as pd
import numpy as np
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoModelForTokenClassification
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import random


In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:

def sentiment_analysis(text):
    """
    Perform sentiment analysis on the given text using a specific model.

    Args:
        text (str): The input text for sentiment analysis.

    Returns:
        list or None: A list of potential human-like sentiments if successful, None otherwise.
    """
    if not text:
        raise ValueError("Input text cannot be empty or None.")

    try:
        model_name = "finiteautomata/bertweet-base-sentiment-analysis"
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Tokenize the input text
        inputs = tokenizer(text, return_tensors="pt")

        # Set up sentiment analysis pipeline
        specific_model = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer)

        # Perform sentiment analysis
        result = specific_model(text)
        # print(f"Result: {result}")

        # Extract sentiment and confidence from the result
        sentiment = result[0]['label']
        confidence = result[0]['score']

        # Map the sentiment label to human-like responses
        sentiment_mappings = {
            'NEG': ['It seems like there are some concerns with the content.', 'The text indicates a negative sentiment.'],
            'POS': ['Looks like a positive message!', 'The text reflects a positive sentiment.'],
            # Add more labels as needed
        }

        predicted_sentiments = random.choice(sentiment_mappings.get(sentiment))

        # Return the list of potential human-like sentiments
        return predicted_sentiments, confidence

    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [130]:
# Load the saved model and tokenizer
model_path = '/content/drive/MyDrive/I2A/LLM/spotify_model'
tokenizer_path = '/content/drive/MyDrive/I2A/LLM/spotify_tokenizer'

model_music = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_music.to(device)

# Prediction function
def music_style(text):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Put the input_ids and attention_mask on the GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Make the prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Map the prediction with labels
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    # Replace this mapping with your actual label mapping
    label_mapping = {0: 'Rock', 1: 'Pop', 2: 'Hip Hop', 3: 'Electronic'}

    # Return the predicted genre
    predicted_genre = label_mapping.get(predicted_label, 'Unknown')
    return predicted_genre
    # # Tokenize input text
    # inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    # # Make prediction
    # with torch.no_grad():
    #     outputs = model_music(**inputs)

    # logits = outputs.logits
    # probabilities = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()

    # # Get the predicted label
    # predicted_label = np.argmax(probabilities)

    # return predicted_label, probabilities


In [127]:
def ner(text):
  # Load the tokenizer and model
  tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
  model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased-finetuned-conll03-english")

  # Tokenize the text
  inputs = tokenizer(text, return_tensors="pt")

  # Predict the entities
  outputs = model(**inputs)
  predictions = outputs[0].argmax(dim=2)

  # Return the entities
  entities = []
  for i, prediction in enumerate(predictions):
    if prediction != -1:
      entities.append(tokenizer.convert_ids_to_tokens(prediction))

  # Return the entities
  return entities

In [134]:
def main():
    # Get the user input
    input_text = input("Enter your query: ")

    # Detect the task
    if re.search(r"sentiment|love|hate|like|dislike|help", input_text, re.IGNORECASE):
        task = "sentiment-analysis"
    elif re.search(r"track_name", input_text, re.IGNORECASE):
        task = "music-style"
    elif re.search(r"ner", input_text, re.IGNORECASE):
        task = "ner"
    else:
        task = "unknown"

    # Perform the task
    if task == "sentiment-analysis":
        sentiment = sentiment_analysis(input_text)
        print("Sentiment:", sentiment[0])
    elif task == "music-style":
        title = re.search(r"track_name:\s+(.+)", input_text).group(1) if re.search(r"track_name:\s+(.+)", input_text) else None
        print(title)
        predicted_genre = music_style(title)
        print(f"Predicted Genre: {predicted_genre}")
    elif task == "ner":
        entities = ner(input_text)
        print("Entities:", entities)
    else:
        print("Unknown task.")

if __name__ == "__main__":
    main()

Enter your query: ner new york


OSError: ignored